In [ ]:
from __future__ import annotations

from modelbase2 import Model, Simulator
from example_models import get_linear_chain_2v, get_poolman2000
from example_models.shared import constant


m = get_linear_chain_2v()
# print(m.get_full_concs({"x": 2.0, "y": 1.0}), "\n")
# print(m.get_args({"x": 2.0, "y": 1.0}), "\n")
# print(m.get_fluxes({"x": 2.0, "y": 1.0}), "\n")
# print(m.get_right_hand_side({"x": 2.0, "y": 1.0}), "\n")

Simulator(m, {"x": 2.0, "y": 1.0}).simulate(10)
Simulator(m, {"x": 2.0, "y": 1.0}).simulate_to_steady_state()

v = Simulator(m, {"x": 2.0, "y": 1.0}).simulate(10).get_fluxes()
c, v = Simulator(m, {"x": 2.0, "y": 1.0}).simulate(10).get_concs_and_fluxes()
c, v = Simulator(m, {"x": 2.0, "y": 1.0}).simulate(10).get_full_concs_and_fluxes()

In [ ]:
from modelbase2 import plot


def moiety_1(x1: float, total: float) -> float:
    return total - x1


m = (
    Model()
    .add_variables({"ATP": 1.0})
    .add_parameters({"ATP_total": 1.0})
    .add_derived("ADP", moiety_1, ["ATP", "ATP_total"])
    .add_reaction("ATPase", constant, {"ATP": -1}, ["ATP"])
)

c, v = Simulator(m).simulate(10).get_full_concs_and_fluxes()
if c is not None:
    plot.line(c)

In [ ]:
s = Simulator(get_poolman2000())

In [ ]:
%%prun

_ = s.simulate(1000).get_full_concs_and_fluxes()

In [ ]:
#          103460 function calls (102810 primitive calls) in 0.045 seconds

#    Ordered by: internal time

#    ncalls  tottime  percall  cumtime  percall filename:lineno(function)
#      1276    0.003    0.000    0.016    0.000 series.py:1261(__setitem__)
# 22709/22701    0.002    0.000    0.003    0.000 {built-in method builtins.isinstance}
#      1284    0.002    0.000    0.002    0.000 cast.py:1760(np_can_hold_element)
#      2560    0.001    0.000    0.002    0.000 base.py:3777(get_loc)
#        29    0.001    0.000    0.027    0.001 model.py:619(_get_rhs)
#      1276    0.001    0.000    0.005    0.000 series.py:1095(__getitem__)
#      2579    0.001    0.000    0.002    0.000 indexing.py:2765(check_dict_or_set_indexers)
#       2/1    0.001    0.001    0.020    0.020 int_assimulo.py:43(integrate)
#      1276    0.001    0.000    0.004    0.000 base.py:341(setitem_inplace)
#      1276    0.001    0.000    0.006    0.000 managers.py:2021(setitem_inplace)
#      1276    0.001    0.000    0.003    0.000 series.py:1220(_get_value)
#      1305    0.001    0.000    0.001    0.000 generic.py:6284(__getattr__)
#      1276    0.001    0.000    0.002    0.000 series.py:1484(_check_is_chained_assignment_possible)
#      1276    0.001    0.000    0.007    0.000 series.py:1369(_set_with_engine)